# [MVTorch](https://github.com/ajhamdi/mvtorch) 3D Calssification Example

## Setup

- install `mvtorch` from [here](https://github.com/ajhamdi/mvtorch/blob/main/INSTALL.md) and activate the environment in the notebook.

- download common 3D datasets ([ModelNet40](https://drive.google.com/uc?export=download&id=157W0qYR2yQAc5qKmXlZuHms66wmUM8Hi), [ScanObjectNN](https://drive.google.com/uc?export=download&id=15xhYA8SC5EdLKZA_xV0FXyRy8f-qGMs5)) and unzip inside `data` directory.

In [1]:
!cd .. && cd .. && cd data/ 
# # download ModelNet40 from https://mega.nz/file/mm5FhJ7I#jGECWn-QSCLH9LLoxhZzSWnf9LCtCavV12toj9SJKPM 
# # download ScanObjectNN from https://mega.nz/file/ampg2QyT#Exo22r-8jzgCa2MOqoqipd39HVqYKG5iykJ5bovjsuI 

## Depenenancies

In [3]:
import sys
import os

import torch
from mvtorch.data import ScanObjectNN, CustomDataLoader
from mvtorch.networks import MVNetwork
from mvtorch.view_selector import MVTN
from mvtorch.mvrenderer import MVRenderer

## Config variables

In [4]:
data_dir='../data/ScanObjectNN' # specifiy where did you put the data rel
nb_views = 1 # Number of views generated by view selector
epochs = 100


## Create dataset and dataloader

In [5]:
dset_train = ScanObjectNN(data_dir=data_dir, split='train')
dset_test = ScanObjectNN(data_dir=data_dir, split='test')
train_loader = CustomDataLoader(dset_train, batch_size=5, shuffle=True, drop_last=False)
test_loader = CustomDataLoader(dset_test, batch_size=5, shuffle=False, drop_last=False)

## define main components 

In [6]:
# Create backbone multi-view network (ResNet18)
mvnetwork = MVNetwork(num_classes=len(dset_train.classes), num_parts=None, mode='cls', net_name='resnet18').cuda()

# Create backbone optimizer
optimizer = torch.optim.AdamW(mvnetwork.parameters(), lr=0.00001, weight_decay=0.03)

# Create view selector
mvtn = MVTN(nb_views=nb_views).cuda()

# Create optimizer for view selector (In case views are not fixed, otherwise set to None)
# mvtn_optimizer = torch.optim.AdamW(mvtn.parameters(), lr=0.0001, weight_decay=0.01)
mvtn_optimizer = None

# Create multi-view renderer
mvrenderer = MVRenderer(nb_views=nb_views, return_mapping=False)

# Create loss function for training
criterion = torch.nn.CrossEntropyLoss()

Using cache found in /home/hamdiaj/.cache/torch/hub/pytorch_vision_v0.8.2


## train/test loop

In [7]:
for epoch in range(epochs):
    correct = 0.0
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("\n\nTraining...")
    mvnetwork.train()
    mvtn.train()
    mvrenderer.train()
    running_loss = 0
    for i, (targets, meshes, points) in enumerate(train_loader):
        azim, elev, dist = mvtn(points, c_batch_size=len(targets))
        rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
        outputs = mvnetwork(rendered_images)[0]

        loss = criterion(outputs, targets.cuda())
        running_loss += loss.item()
        loss.backward()
        correct += (torch.max(outputs, dim=1)[1] == targets.cuda()).to(torch.int32).sum().item()
        optimizer.step()
        optimizer.zero_grad()
        if mvtn_optimizer is not None:
            mvtn_optimizer.step()
            mvtn_optimizer.zero_grad()
        
        if (i + 1) % int(len(train_loader) * 0.25) == 0:
            print(f"\tBatch {i + 1}/{len(train_loader)}: Current Average Training Loss = {(running_loss / (i + 1)):.5f}")
    print(f"\nAverage Training Loss = {(running_loss / len(train_loader)):.5f}. Average Training Accuracy = {(100.0*correct / len(dset_train)):.2f}.")

    print("\n\nTesting...")
    mvnetwork.eval()
    mvtn.eval()
    mvrenderer.eval()
    running_loss = 0
    correct = 0.0
    for i, (targets, meshes, points) in enumerate(test_loader):
        with torch.no_grad():
            azim, elev, dist = mvtn(points, c_batch_size=len(targets))
            rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
            outputs = mvnetwork(rendered_images)[0]

            loss = criterion(outputs, targets.cuda())
            running_loss += loss.item()
            correct += (torch.max(outputs, dim=1)[1] == targets.cuda()).to(torch.int32).sum().item()

            if (i + 1) % int(len(test_loader) * 0.25) == 0:
                print(f"\tBatch {i + 1}/{len(test_loader)}: Current Average Test Loss = {(running_loss / (i + 1)):.5f}")
    print(f"\nTotal Average Test Loss = {(running_loss / len(test_loader)):.5f}.  Average Test Accuracy = {(100.0*correct / len(dset_test)):.2f}.")


Epoch 1/100


Training...
	Batch 116/464: Current Average Training Loss = 2.69195
	Batch 232/464: Current Average Training Loss = 2.52120
	Batch 348/464: Current Average Training Loss = 2.42160
	Batch 464/464: Current Average Training Loss = 2.33012

Average Training Loss = 2.33012. Average Training Accuracy = 25.61.


Testing...
	Batch 29/117: Current Average Test Loss = 2.07354
	Batch 58/117: Current Average Test Loss = 1.85165
	Batch 87/117: Current Average Test Loss = 1.87505
	Batch 116/117: Current Average Test Loss = 1.89656

Total Average Test Loss = 1.90131.  Average Test Accuracy = 44.77.

Epoch 2/100


Training...
	Batch 116/464: Current Average Training Loss = 2.01555
	Batch 232/464: Current Average Training Loss = 1.93305


KeyboardInterrupt: 

### [OPTIONAL] visualize results 